In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

# Setup WebDriver with additional options
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 30)


url_dic = {
    'Kerala': 'https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile',
    'Telangana': 'https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile',
    'Bihar': 'https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile',
    'West Bengal': 'https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile',
    'Jammu&Kashmir': 'https://www.redbus.in/online-booking/jkrtc',
    'Chandigarh': 'https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu',
    'Assam': 'https://www.redbus.in/online-booking/astc/?utm_source=rtchometile',#
    'Rajasthan ': 'https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile',
    'Himachal Pradesh': 'https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile',#
    'Andhra Pradesh': 'https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile'
}
bus_data = []

for state, url in url_dic.items():
    driver.get(url)
    time.sleep(5)

    page = 1
    all_routes = []

    while True:
        print(f"[{state}] Processing route page: {page}")

        try:
            nested_div = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]')))
            route_list = nested_div.find_elements(By.CLASS_NAME, 'route_link')

            if not route_list:
                print("No more routes found.")
                break

            for route in route_list:
                link_element = route.find_element(By.TAG_NAME, 'a')
                route_name = link_element.get_attribute('title')
                route_link = link_element.get_attribute('href')
                all_routes.append((state, route_name, route_link))

            next_buttons = driver.find_elements(By.CSS_SELECTOR, ".DC_117_pageTabs")
            if page >= len(next_buttons):
                print("No more route pages.")
                break

            next_button = next_buttons[page]
            driver.execute_script("arguments[0].click();", next_button)
            page += 1
            time.sleep(2)

        except Exception as e:
            print(f"Error processing route page {page} for {state}: {e}")
            break

    for state, route_name, route_link in all_routes:
        driver.get(route_link)
        time.sleep(3)

        try:
            view_buses_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".button")))
            view_buses_button.click()
            time.sleep(3)
        except Exception as e:
            print(f"Error clicking 'View Buses' button for {route_name}: {e}")
            continue

        for i in range(5):  # Scroll 5 times
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)

        try:
            bus_cards = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))
            if not bus_cards:
                print(f"No bus cards found for route: {route_name}")
                continue
            print(f"Found {len(bus_cards)} buses for route: {route_name}")
        except Exception as e:
            print(f"Error loading bus cards for {route_name}: {e}")
            continue

        for bus in bus_cards:
            try:
                bus_name = bus.find_element(By.CLASS_NAME, "travels").text
                bus_type = bus.find_element(By.CLASS_NAME, "bus-type").text
                departure_time = bus.find_element(By.CLASS_NAME, "dp-time").text
                duration = bus.find_element(By.CLASS_NAME, "dur").text
                reaching_time = bus.find_element(By.CLASS_NAME, "bp-time").text

                rating_element = bus.find_elements(By.CSS_SELECTOR, ".rating-sec .rating span")
                star_rating = rating_element[0].text if len(rating_element) > 0 else "N/A"

                fare_element = bus.find_elements(By.CLASS_NAME, "fare")
                fare = fare_element[0].text.split()[-1] if len(fare_element) > 0 and len(fare_element[0].text.split()) > 0 else "N/A"

                seat_element = bus.find_elements(By.CLASS_NAME, "seat-left")
                if len(seat_element) > 0:
                    seats_available = seat_element[0].text.split()[0] if seat_element[0].text else "N/A"
                else:
                    seats_available = "N/A"

                bus_data.append({
                    "state": state,
                    "route_name": route_name,
                    "route_link": route_link,
                    "busname": bus_name,
                    "bustype": bus_type,
                    "departing_time": departure_time,
                    "duration": duration,
                    "reaching_time": reaching_time,
                    "star_rating": star_rating,
                    "price": fare,
                    "seats_available": seats_available
                })

            except Exception as e:
                print(f"Error extracting data for bus: {e}")
                continue

# Save results to CSV
with open("bus_datas.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=[
        "state", "route_name", "route_link", "busname", "bustype",
        "departing_time", "duration", "reaching_time", "star_rating", "price", "seats_available"
    ])
    writer.writeheader()
    writer.writerows(bus_data)

driver.quit()


[Kerala] Processing route page: 1
No more route pages.
Found 56 buses for route: Bangalore to Kozhikode
Found 10 buses for route: Kozhikode to Ernakulam
Found 10 buses for route: Kozhikode to Bangalore
Found 10 buses for route: Ernakulam to Kozhikode
Found 10 buses for route: Kozhikode to Mysore
Found 10 buses for route: Mysore to Kozhikode
Found 10 buses for route: Kozhikode to Thrissur
Found 10 buses for route: Thrissur to Kozhikode
Found 10 buses for route: Bangalore to Kannur (Kerala)
[Telangana] Processing route page: 1
[Telangana] Processing route page: 2
No more route pages.
Found 10 buses for route: Khammam to Hyderabad
Found 5 buses for route: Hyderabad to Srisailam
Found 10 buses for route: Hyderabad to Khammam
Found 10 buses for route: Hyderabad to Vijayawada
Found 10 buses for route: Karimnagar to Hyderabad
Found 10 buses for route: Hyderabad to Mancherial
Found 10 buses for route: Hyderabad to Adilabad
Found 10 buses for route: Hyderabad to Nirmal
Found 10 buses for route:

In [16]:
df=pd.read_csv("bus_datas.csv")

state                0
route_name           0
route_link           0
busname            517
bustype            547
departing_time     517
duration           517
reaching_time      517
star_rating        542
price              518
seats_available    536
dtype: int64

In [17]:
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)


In [15]:
 a

,state,route_name,route_link,busname,bustype,departing_time,duration,reaching_time,star_rating,price,seats_available
0,Kerala,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 581,Super Express Non AC Seater Air Bus (2+2),10:00,07h 45m,17:45,4.0,667.0,28.0
1,Kerala,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 52,SWIFT-GARUDA A/C SEATER BUS,12:00,08h 05m,20:05,3.4,863.0,32.0
2,Kerala,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 3922,SWIFT-HYBRID NON AC SEATER CUM SLEEPER,12:55,07h 50m,20:45,3.8,677.0,22.0
3,Kerala,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 3848,AC MULTI AXLE,13:47,07h 39m,21:26,2.4,986.0,29.0
4,Kerala,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,10h 10m,01:10,3.6,742.0,38.0
...,...,...,...,...,...,...,...,...,...,...,...
1691,Andhra Pradesh,Visakhapatnam to Kakinada,https://www.redbus.in/bus-tickets/visakhapatna...,APSRTC - 23892,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",00:25,04h 02m,04:27,3.7,281.0,8.0
1692,Andhra Pradesh,Visakhapatnam to Kakinada,https://www.redbus.in/bus-tickets/visakhapatna...,APSRTC - 2768,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",01:00,04h 00m,05:00,3.7,281.0,33.0
1700,Andhra Pradesh,Bangalore to Madanapalli,https://www.redbus.in/bus-tickets/bangalore-to...,APSRTC - 7591,Express Non AC Seater 2+3,05:45,03h 15m,09:00,4.6,173.0,38.0
1701,Andhra Pradesh,Bangalore to Madanapalli,https://www.redbus.in/bus-tickets/bangalore-to...,APSRTC - 6523,"ULTRA DELUXE (NON-AC, 2+2 PUSH BACK)",07:45,03h 15m,11:00,4.1,226.0,34.0


In [18]:
df

,state,route_name,route_link,busname,bustype,departing_time,duration,reaching_time,star_rating,price,seats_available
0,Kerala,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 581,Super Express Non AC Seater Air Bus (2+2),10:00,07h 45m,17:45,4.0,667.0,28.0
1,Kerala,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 52,SWIFT-GARUDA A/C SEATER BUS,12:00,08h 05m,20:05,3.4,863.0,32.0
2,Kerala,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 3922,SWIFT-HYBRID NON AC SEATER CUM SLEEPER,12:55,07h 50m,20:45,3.8,677.0,22.0
3,Kerala,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 3848,AC MULTI AXLE,13:47,07h 39m,21:26,2.4,986.0,29.0
4,Kerala,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,10h 10m,01:10,3.6,742.0,38.0
...,...,...,...,...,...,...,...,...,...,...,...
1143,Andhra Pradesh,Visakhapatnam to Kakinada,https://www.redbus.in/bus-tickets/visakhapatna...,APSRTC - 23892,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",00:25,04h 02m,04:27,3.7,281.0,8.0
1144,Andhra Pradesh,Visakhapatnam to Kakinada,https://www.redbus.in/bus-tickets/visakhapatna...,APSRTC - 2768,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",01:00,04h 00m,05:00,3.7,281.0,33.0
1145,Andhra Pradesh,Bangalore to Madanapalli,https://www.redbus.in/bus-tickets/bangalore-to...,APSRTC - 7591,Express Non AC Seater 2+3,05:45,03h 15m,09:00,4.6,173.0,38.0
1146,Andhra Pradesh,Bangalore to Madanapalli,https://www.redbus.in/bus-tickets/bangalore-to...,APSRTC - 6523,"ULTRA DELUXE (NON-AC, 2+2 PUSH BACK)",07:45,03h 15m,11:00,4.1,226.0,34.0
